In [1]:
import os
import fnmatch

In [35]:
def get_ignore_patterns(start_path):
    patterns = []
    
    # Read all .gitignore files
    for root, dirs, files in os.walk(start_path):
        if '.gitignore' in files:
            with open(os.path.join(root, '.gitignore'), 'r') as f:
                for line in f.readlines():
                    line = line.strip()  # remove whitespace
                    if line and not line.startswith('#'):  # ignore empty lines and comments
                        # Add the root to the pattern to correctly match subdirectories
                        pattern = os.path.join(root, line)
                        patterns.append(pattern)    
    patterns.append('./*.ipynb')
    return patterns

def get_file_structure(start_path, ignore_paths):
    file_structure = {}
    ignore_patterns = get_ignore_patterns(start_path)
    for root, dirs, files in os.walk(start_path, topdown=True):
        dirs[:] = [d for d in dirs if d not in ignore_paths]  # remove the ignored directories from dirs
        files[:] = [f for f in files if f not in ignore_paths]  # remove the ignored directories from dirs        
        for file in files:
            path = os.path.join(root, file)
            if not any(fnmatch.fnmatch(path, pattern) for pattern in ignore_patterns):  # only add if not in ignore list
                current_level = file_structure
                path_parts = path.split(os.sep)[1:]  # split the path into parts and ignore the first part (the start_path)
                for part in path_parts[:-1]:  # for all parts except the last one (the file itself)
                    current_level = current_level.setdefault(part, {})
                current_level[path_parts[-1]] = None  # set the file as a key with value None
    return file_structure

def print_file_structure(structure, descriptions, indent_level=0):
    indent = '  ' * indent_level  # two spaces per indent level
    result = ""
    for name, substructure in structure.items():
        if substructure is None:  # it's a file
            description = descriptions.get(name, '')
            if description == '':
                result += "{}- {}\n".format(indent, name)
#                 print("{}- {}".format(indent, name))
            else:
                result += "{}- {}: {}\n".format(indent, name, description)
#                 print("{}- {}: {}".format(indent, name, description))
        else:  # it's a directory
            result += f'{indent}- {name}/\n'
#             print(f'{indent}- {name}/')
            result += print_file_structure(substructure, descriptions, indent_level+2)
    return result


In [9]:
description = {
    "main.py" : "a Flask server Script that handles user registration, login, and interactive chat sessions using SocketIO, with a chatbot utilizing GPT for generating responses.",
    "simple_gpt_chatbot.py" : "Interfaces with OpenAI's GPT-3.5 model to generate responses for a chatbot, using the historical data of a chat session retrieved from a database.",
    "model.py" : "Defines the structure of the database tables for the chat application, including Users, ChatSessions, ChatMessages, Chatbots, and the relationships between them.",
    "database.py" : "Initializes an SQLAlchemy database instance for the application to interact with the database.",
    "App.js" : "A React-based client-side application script managing user authentication, registration, chat sessions using Socket.IO, and component rendering according to user authentication status.",
    "Chat.js" : "A React component managing the real-time chat interface, handling message sending/receiving via Socket.IO, maintaining chat session state, and rendering the chat messages and input field." ,
    "Input.js" : "A React component that renders a message input field and send button, handling message submission and maintaining the state of the typed message.",
    "Login.js" : "A React component that renders a login form with input fields for ID and passcode, handles form submission, and performs login request to the server using axios.",
    "Message.js" : "A React component that renders a chat message, including the profile photo, username (for remote messages), and the content of the message.",
    "Registration.js" : "A React component that renders a registration form with input fields for ID and passcode, handles form submission, performs registration request to the server using axios, and subsequently performs login.",
    "SocketContext.js" : "A React context that provides a Socket.IO socket instance to its child components.",
    "index.js" : "The entry point of the React application, rendering the root component (App) into the DOM, enabling React Strict Mode, and optionally measuring performance using the reportWebVitals function.",    
}

In [36]:
start_path = "."
ignore_files = ["venv", "node_modules", "build", ".git", "logo.svg", "public", "Procfile", ".ipynb_checkpoints", "LICENSE", "README.md", "copilot.ipynb", ".gitignore", ".env", ".DS_Store"]
# "package-lock.json", "package.json"
file_structure = get_file_structure(start_path, ignore_files)
file_descriptions = print_file_structure(file_structure, description, 0)

In [62]:
project_name = "nplace_webchat"


role = """
Your role is developer and product designer copilot. We collaborate on project: {}.
Use your knowledge and capabilities to assist me in the best possible manner, keeping in mind that I am unfamiliar with many aspects of software development.
""".format(project_name)


project_overview = """
Develop a website enabling user registration/login and chat with AI-powered chatbots.
Users can create, rename, and delete personalized chat sessions.
When creating a chat session, user can choose from a list of predefined themes, such as dragon and dungeon, mafia, etc.
Depending on the theme, session may feature one or multiple chatbots, but restricted to a single user.
Future work will explore instant messaging between multiple users.

"""


steps_taken = """
- Simple frontend using React with Messenger-like UI, backend using Flask
- Code on Github, hosted on Railway, no CICD 
- Simple user authentication using JWT
- PostgreSQL DB to store user credentials and chat data
- Simple chatbot using OpenAI GPT API
"""

prompt_1 = f"""{role}
Project overview {project_overview}
Steps taken {steps_taken}
Current files
{file_descriptions}
Show that you understand the overview and current status. If unclear, ask questions. Next prompt, I will brief you details about current task."""

print(prompt_1)


Your role is developer and product designer copilot. We collaborate on project: nplace_webchat.
Use your knowledge and capabilities to assist me in the best possible manner, keeping in mind that I am unfamiliar with many aspects of software development.

Project overview 
Develop a website enabling user registration/login and chat with AI-powered chatbots.
Users can create, rename, and delete personalized chat sessions.
When creating a chat session, user can choose from a list of predefined themes, such as dragon and dungeon, mafia, etc.
Depending on the theme, session may feature one or multiple chatbots, but restricted to a single user.
Future work will explore instant messaging between multiple users.


Steps taken 
- Simple frontend using React with Messenger-like UI, backend using Flask
- Code on Github, hosted on Railway, no CICD 
- Simple user authentication using JWT
- PostgreSQL DB to store user credentials and chat data
- Simple chatbot using OpenAI GPT API

Current files
- 

In [70]:
current_task = """
- Each chat session can be renamed, deleted
- A button to create new chat sessions
- When a chat session is selected, past history in this session is loaded and displayed
- The list is sorted by latest view, with the session latest selected showing on bottom
"""


duties = """
- Refine and articulate features
- Assist my design decision by list options with pros and cons
- Break down implementation into steps and walk me through
"""


misc = """

"""

prompt_2 = f"""Current task{current_task}
Proposed steps for you to help{duties}
Be concise and professional. Ask question whenever things get unclear. 
Do step by step, don't dump info. For example, first show me which files need update to add feature, then one file at time ask me to paste the file to you and show me the changes to make.
After a step, ask me to proceed.
"""

print(prompt_2)

Current task
- Each chat session can be renamed, deleted
- A button to create new chat sessions
- When a chat session is selected, past history in this session is loaded and displayed
- The list is sorted by latest view, with the session latest selected showing on bottom

Proposed steps for you to help
- Refine and articulate features
- Assist my design decision by list options with pros and cons
- Break down implementation into steps and walk me through

Be concise and professional. Ask question whenever things get unclear. 
Do step by step, don't dump info. For example, first show me which files need update to add feature, then one file at time ask me to paste the file to you and show me the changes to make.



In [ ]:
qna = """
How are the themes for the chat sessions implemented? Are these themes influencing the chatbot behavior or is it just a cosmetic detail for the user interface?
I plan to use Python and prompt engineering to alter chatbot behavior on server side.
How is the chatbot's state managed across different sessions? Are there any rules or constraints regarding how a chatbot should behave in different sessions or with different users?
Not decided yet.
In the future, when you plan to explore instant messaging between multiple users, will the chatbot still participate in these sessions or will it be strictly user-to-user?
Chatbots still participate.



"""